In [1]:
import gym
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make('MountainCarContinuous-v0')
env.seed(1)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


[1]

In [2]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(2,8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)
        self.fc3_ = nn.Linear(4, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = self.fc3(x)
        sigma_ = self.fc3_(x)
        return mu, sigma_

In [3]:
policy = PolicyNetwork()
optimizer = optim.Adam(policy.parameters(), lr=1e-3)
max_episodes = 1000
gamma = 0.9
reward_all = []
for episode in range(max_episodes):
    rewards = []
    log_probs = []
    state = env.reset()
    while True:
        mu, sigma_ = policy(torch.FloatTensor([state]))
        sigma_ = F.softplus(sigma_) # sigma_ > 0
        action = mu + sigma_.sqrt() * torch.randn(1) # sampling action from normal distribution
        prob = (-1 * (action - mu).pow(2)/(2 * sigma_)).exp() / (2 * sigma_ * math.pi).sqrt()
        log_prob = prob.log()
        action = F.tanh(action)
        state, reward, done, _ = env.step([action.item()])
        rewards.append(reward)
        log_probs.append(log_prob)
#         print(action, reward)
        if done:
            Return = torch.zeros(1,1)
            for reward, log_prob in zip(reversed(rewards), reversed(log_probs)):
                Return = reward + gamma * Return
                loss = - log_prob * Return
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print("Episode: %d, reward %.1f" % (episode, np.sum(rewards)))
            reward_all.append(np.sum(rewards))
            rewards = []
            log_probs = []
            break

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode: 0, reward -38.8
Episode: 1, reward -90.5
Episode: 2, reward -97.0
Episode: 3, reward -98.3
Episode: 4, reward -98.7
Episode: 5, reward -99.3
Episode: 6, reward -99.8
Episode: 7, reward -99.9
Episode: 8, reward -99.9
Episode: 9, reward -99.8
Episode: 10, reward -99.9
Episode: 11, reward -99.9
Episode: 12, reward -99.8
Episode: 13, reward -99.9
Episode: 14, reward -99.9
Episode: 15, reward -99.9
Episode: 16, reward -99.9
Episode: 17, reward -99.9
Episode: 18, reward -99.9
Episode: 19, reward -99.9
Episode: 20, reward -99.9
Episode: 21, reward -99.9
Episode: 22, reward -99.9
Episode: 23, reward -99.9
Episode: 24, reward -99.9
Episode: 25, reward -99.9
Episode: 26, reward -99.9
Episode: 27, reward -99.9
Episode: 28, reward -99.9
Episode: 29, reward -99.9
Episode: 30, reward -99.9
Episode: 31, reward -99.9
Episode: 32, reward -99.9
Episode: 33, reward -99.9
Episode: 34, reward -99.9
Episode: 35, reward -99.9
Episode: 36, reward -99.9
Episode: 37, reward -99.9
Episode: 38, reward -9

Episode: 308, reward -99.9
Episode: 309, reward -99.9
Episode: 310, reward -99.9
Episode: 311, reward -99.9
Episode: 312, reward -99.9
Episode: 313, reward -99.9
Episode: 314, reward -99.9
Episode: 315, reward -99.9
Episode: 316, reward -99.9
Episode: 317, reward -99.9
Episode: 318, reward -99.9
Episode: 319, reward -99.9
Episode: 320, reward -99.9
Episode: 321, reward -99.9
Episode: 322, reward -99.9
Episode: 323, reward -99.9
Episode: 324, reward -99.9
Episode: 325, reward -99.9
Episode: 326, reward -99.9
Episode: 327, reward -99.9
Episode: 328, reward -99.9
Episode: 329, reward -99.9
Episode: 330, reward -99.9
Episode: 331, reward -99.9
Episode: 332, reward -99.9
Episode: 333, reward -99.9
Episode: 334, reward -99.9
Episode: 335, reward -99.9
Episode: 336, reward -99.9
Episode: 337, reward -99.9
Episode: 338, reward -99.9
Episode: 339, reward -99.9
Episode: 340, reward -99.9
Episode: 341, reward -99.9
Episode: 342, reward -99.9
Episode: 343, reward -99.9
Episode: 344, reward -99.9
E

Episode: 612, reward -99.9
Episode: 613, reward -99.9
Episode: 614, reward -99.9
Episode: 615, reward -99.9
Episode: 616, reward -99.9
Episode: 617, reward -99.9
Episode: 618, reward -99.9
Episode: 619, reward -99.9
Episode: 620, reward -99.9
Episode: 621, reward -99.9
Episode: 622, reward -99.9
Episode: 623, reward -99.9
Episode: 624, reward -99.9
Episode: 625, reward -99.9
Episode: 626, reward -99.9
Episode: 627, reward -99.9
Episode: 628, reward -99.9
Episode: 629, reward -99.9
Episode: 630, reward -99.9
Episode: 631, reward -99.9
Episode: 632, reward -99.9
Episode: 633, reward -99.9
Episode: 634, reward -99.9
Episode: 635, reward -99.9
Episode: 636, reward -99.9
Episode: 637, reward -99.9
Episode: 638, reward -99.9
Episode: 639, reward -99.9
Episode: 640, reward -99.9
Episode: 641, reward -99.9
Episode: 642, reward -99.9
Episode: 643, reward -99.9
Episode: 644, reward -99.9
Episode: 645, reward -99.9
Episode: 646, reward -99.9
Episode: 647, reward -99.9
Episode: 648, reward -99.9
E

Episode: 916, reward -99.9
Episode: 917, reward -99.9
Episode: 918, reward -99.9
Episode: 919, reward -99.9
Episode: 920, reward -99.9
Episode: 921, reward -99.9
Episode: 922, reward -99.9
Episode: 923, reward -99.9
Episode: 924, reward -99.9
Episode: 925, reward -99.9
Episode: 926, reward -99.9
Episode: 927, reward -99.9
Episode: 928, reward -99.9
Episode: 929, reward -99.9
Episode: 930, reward -99.9
Episode: 931, reward -99.9
Episode: 932, reward -99.9
Episode: 933, reward -99.9
Episode: 934, reward -99.9
Episode: 935, reward -99.9
Episode: 936, reward -99.9
Episode: 937, reward -99.9
Episode: 938, reward -99.9
Episode: 939, reward -99.9
Episode: 940, reward -99.9
Episode: 941, reward -99.9
Episode: 942, reward -99.9
Episode: 943, reward -99.9
Episode: 944, reward -99.9
Episode: 945, reward -99.9
Episode: 946, reward -99.9
Episode: 947, reward -99.9
Episode: 948, reward -99.9
Episode: 949, reward -99.9
Episode: 950, reward -99.9
Episode: 951, reward -99.9
Episode: 952, reward -99.9
E

In [4]:
class StateValueNetwork(nn.Module):
    def __init__(self):
        super(StateValueNetwork, self).__init__()
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
policy_network = PolicyNetwork()
state_value_network = StateValueNetwork()
optimizer_policy = optim.Adam(policy_network.parameters(), lr=1e-3)
optimizer_state_value = optim.Adam(state_value_network.parameters(), lr=1e-4)
max_episodes = 1000
num_steps = 200
gamma = 0.9
reward_all = []
for episode in range(max_episodes):
    rewards = []
    log_probs = []
    state_values = []
    state = env.reset()
    step = 0
    while True:
        mu, sigma_ = policy(torch.FloatTensor([state]))
        sigma_ = F.softplus(sigma_) # sigma_ > 0
        action = mu + sigma_.sqrt() * torch.randn(1) # sampling action from normal distribution
        prob = (-1 * (action - mu).pow(2)/(2 * sigma_)).exp() / (2 * sigma_ * math.pi).sqrt()
        log_prob = prob.log()
        action = F.tanh(action)
        state_value = state_value_network(torch.FloatTensor([state]))
        state, reward, done, _ = env.step([action.item()])
        rewards.append(reward)
        log_probs.append(log_prob)
        state_values.append(state_value)
        step += 1
        
        if step >= num_steps or done:
            Return = torch.zeros(1,1)
            if done:
                next_state_value = torch.zeros(1,1)
            else:
                mu, sigma_ = policy(torch.FloatTensor([state]))
                sigma_ = F.softplus(sigma_) # sigma_ > 0
                action = mu + sigma_.sqrt() * torch.randn(1)
                action = F.tanh(action)
                state, reward, done, _ = env.step([action.item()])
                next_state_value = state_value_network(torch.FloatTensor([state]))
            for reward, log_prob, state_value in zip(reversed(rewards), reversed(log_probs), reversed(state_values)):
                Return = reward + gamma * Return
                advantage = Return + (gamma ** step) * next_state_value - state_value
                policy_loss = - log_prob * advantage
                
                optimizer_state_value.zero_grad()
                optimizer_policy.zero_grad()
                state_value_loss = advantage.pow(2) ## Advantage * Advantage ??? Advantage doesn't work!
                state_value_loss.backward(retain_graph=True)
                policy_loss.backward(retain_graph=True)
                optimizer_state_value.step()
                optimizer_policy.step()
        if done:
            print("Episode: %d, reward %.1f" % (episode, np.sum(rewards)))
            reward_all.append(np.sum(rewards))
            rewards = []
            log_probs = []
            break

KeyboardInterrupt: 